##### 1. Packages
**For, collab, uncomment and install the packages specified below**

In [ ]:
# !pip3 install torch torchvision torchaudio

In [ ]:
# !pip install stable-baselines3[extra] protobuf==3.20.*

In [ ]:
# !pip install mss pydirectinput pytesseract

##### 1. Import the dependencies
**For, collab, uncomment and install the packages specified below**

In [1]:
from mss import mss # used for screen capture 
import pydirectinput # used for keyboard input
import cv2 #used for image processing
import numpy as np 
import pytesseract #used for text recognition
from matplotlib import pyplot as plt
import time
from gym import Env # used for creating custom environment
from gym.spaces import Box, Discrete # used for defining action and observation space

ModuleNotFoundError: No module named 'gym'

#### 2. Build the Environment

2.1 Create Environment